In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import joblib
emb_matrix_train = joblib.load("/content/drive/MyDrive/sentipol_embedd_7410_128_768_alberto_training.joblib")
wmal_matrix_train = joblib.load("/content/drive/MyDrive/wmal_training_7410_128.joblib")

In [ ]:
import pandas as pd
dataset_DF = pd.read_csv("/content/drive/MyDrive/sentipolc_train_processed.csv")

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = dataset_DF["oneg"]
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

In [ ]:
from keras.layers import Input, Dense, LSTM, Bidirectional, TimeDistributed, Flatten, Concatenate
!pip install keras-self-attention

In [ ]:
from keras_self_attention import SeqWeightedAttention

In [ ]:
import keras
input_emb = Input(shape=(128,768))
input_wmal = Input(shape=(128,))

#Bi-LSTM
bi = Bidirectional(LSTM(32, return_sequences=True))(input_emb)
att = SeqWeightedAttention()(bi)
flat = Flatten()(att)

#WMAL 
d1 = Dense(64) (input_wmal)

#Concat
concat = Concatenate(axis=1)([flat, d1])

d2 = Dense(32) (concat)
output = Dense(2,activation="softmax") (d2)

m = keras.Model(inputs=[input_emb,input_wmal], outputs=output)
m.compile(loss="categorical_crossentropy",optimizer='adam',metrics=["accuracy"])





In [ ]:
m.summary()

In [ ]:
m.fit([emb_matrix_train,wmal_matrix_train],onehot_encoded,batch_size=128, epochs=8)

In [ ]:
emb_matrix_test = joblib.load("/content/drive/MyDrive/sentipol_embedd_2000_128_768_alberto_test.joblib")
dataset_test = pd.read_csv("/content/drive/MyDrive/sentipolc_test_processed.csv")
wmal_matrix_test = joblib.load("/content/drive/MyDrive/wmal_test_2000_128.joblib")

In [ ]:
# define example
data = dataset_test["oneg"]
values = array(data)
print(values)
# integer encode
integer_encoded_test = label_encoder.transform(values)
print(integer_encoded_test)
# binary encode
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoded_test = onehot_encoder.transform(integer_encoded_test)
print(onehot_encoded_test)

In [ ]:
predictions = m.predict([emb_matrix_test,wmal_matrix_test])

In [ ]:
print(predictions)

In [ ]:
import numpy as np
predictions_vec = np.zeros((2000,4))

In [ ]:
i = 0
confidences= []
for item in predictions:
  index = np.argmax(item);
  confidences.append(item[index])
  predictions_vec[i][index] = 1
  i = i+1

In [ ]:
labels_to_eval = []
for vec in predictions_vec:
  inverted = label_encoder.inverse_transform([argmax(vec)])[0]
  labels_to_eval.append(inverted)

In [ ]:
confidences

In [ ]:
len(labels_to_eval)

In [ ]:
dataset_test["predicted"] = labels_to_eval
dataset_test["confidence"] = confidences

In [ ]:
!pip install scikit-learn

from sklearn.metrics import classification_report
print(classification_report(dataset_test["oneg"],labels_to_eval,digits=5))

In [ ]:
dataset_test

In [ ]:
m.save("/content/drive/MyDrive/model_1_neg_sentipolc_0.___.h5")

In [ ]:
dataset_test.to_csv("/content/drive/MyDrive/predictions_SENTIPOC_test_neg_full.csv");